In [30]:
from ctgan import load_demo
from ctgan import CTGAN
import pandas as pd

from sklearn.preprocessing import StandardScaler


In [127]:
df_m_0 = "min_0"
df_m_1 = "min_1"
df_m_2 = 'min_2'
# df_m_3 = pd.read_csv('../dataset/min_3.csv')
df_0 = 'informative_cluster_0'
df_1 = 'informative_cluster_1'
# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_1
df= pd.read_csv("../dataset/" + selected_file + ".csv")

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6509 entries, 0 to 6508
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      6509 non-null   int64  
 1   type            6509 non-null   int64  
 2   amount          6509 non-null   float64
 3   oldbalanceOrg   6509 non-null   float64
 4   newbalanceOrig  6509 non-null   float64
 5   oldbalanceDest  6509 non-null   float64
 6   newbalanceDest  6509 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 356.1 KB


In [129]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,0,9442.86,0.00,9442.86,89555.86,80113.01
1,0,10485.04,10411.00,20896.04,119631.29,109146.25
2,0,7116.65,228235.00,235351.65,122203.77,115087.12
3,0,6200.80,126034.58,132235.38,611206.60,605005.80
4,0,14965.91,18467.00,33432.91,316691.28,301725.37


In [130]:
from sdv.cag import SingleTableProgrammableConstraint

class TransactionBalanceLogic(SingleTableProgrammableConstraint):
    def is_valid(self, row):
        print(row)
        amount = row['amount']
        obo = row['oldbalanceOrg']
        nbo = row['newbalanceOrig']
        obd = row['oldbalanceDest']
        nbd = row['oldbalanceDest']

        # if any(v is None for v in [t, obo, nbo, obd]):
        #     return True  # skip NaNs
        return (obd >= 0) & (obo >= 0) & (nbo >= 0) & (amount>0) & (nbd >= 0)

    def transform(self, table_data):
        return table_data  # no transformation needed

    def get_updated_metadata(self, metadata):
        new_metadata = metadata.copy()
        return new_metadata

    def reverse_transform(self, table_data):
        return table_data


In [131]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata
metadata = Metadata.detect_from_dataframe(df)
synthesizer = CTGANSynthesizer(metadata)
synthesizer.add_constraints([TransactionBalanceLogic()])
synthesizer.fit(df)

I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdv\single_table\base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [132]:
data = synthesizer.sample(df.shape[0],output_file_path="../dataset/fake_" + selected_file + ".csv")

      type     amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0        0   16318.37        2782.07          543.40          900.86   
1        1    8673.88       20475.46        12087.16        17774.73   
2        3   14742.25       41266.93       180973.42         4759.28   
3        1  458282.69       27077.05         3053.57       186138.65   
4        3    1732.84      128967.95       436517.62         6278.58   
...    ...        ...            ...             ...             ...   
6504     1    9954.42       63658.35       311671.37         3042.07   
6505     1  147607.18       14341.25        11747.69       136534.54   
6506     3    6134.67        9612.99            0.00            0.00   
6507     0   20686.07      170538.13       202648.34          963.04   
6508     3   10554.80       63238.38       245705.57         2237.95   

      newbalanceDest  
0            7168.14  
1          523596.49  
2               0.00  
3          329157.14  
4            9470.85